In [1]:
import tensorflow as tf
import sounddevice as sd
import numpy as np
import IPython.display as ipd

In [2]:
tf.config.set_visible_devices(tf.config.list_physical_devices()[0])
tf.config.list_logical_devices()

[LogicalDevice(name='/device:CPU:0', device_type='CPU'),
 LogicalDevice(name='/device:XLA_CPU:0', device_type='XLA_CPU')]

In [3]:
sd.default.samplerate = 44100
sd.default.channels = 1

In [32]:
path_to_model = 'models/conv/Mira/MiraConv-v1.3.h5'
model = tf.keras.models.load_model(path_to_model)
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=[
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.AUC(name='auc'),
        tf.keras.metrics.BinaryAccuracy(),
    ]
)
model.summary()

Model: "MiraConv-v1.3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_24 (Conv1D)           (None, 100, 20)           8820      
_________________________________________________________________
re_lu_24 (ReLU)              (None, 100, 20)           0         
_________________________________________________________________
conv1d_25 (Conv1D)           (None, 20, 1)             100       
_________________________________________________________________
re_lu_25 (ReLU)              (None, 20, 1)             0         
_________________________________________________________________
flatten_12 (Flatten)         (None, 20)                0         
_________________________________________________________________
reshape_11 (Reshape)         (None, 1, 20)             0         
_________________________________________________________________
gru_8 (GRU)                  (None, 20)              

In [33]:
data = sd.rec(int(1.0 * sd.default.samplerate), blocking=True)

In [34]:
def callback(indata, outdata, frames, time, status):
    # outdata[:] = indata
    global data, model
    data = data[frames:]
    data = np.append(data, indata, axis=0)
    prediction = model(np.reshape(data, (1, *data.shape)),training=False)[0][0].numpy()
    ipd.clear_output(wait=True)
    if prediction > 0.99:
        sd.play(data)
        print('Mira {:.3f}'.format(prediction))
    else:
        ipd.clear_output()
    
block = int(sd.default.samplerate * 0.1)
stream = sd.Stream(blocksize=block, callback=callback)

In [35]:
stream.start()

In [36]:
stream.stop()

In [37]:
data.shape

(44100, 1)

In [38]:
model.input_shape

(None, 44100, 1)